### 1. Inisialisasi Pemrosesan Dokumen dan Vektor Store

In [ ]:
import os
from IPython.display import display, Markdown
from utils.document_processor import DocumentProcessor
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

## Inisialisasi pemrosesan dokumen
docs = DocumentProcessor()  # Membuat instance dari DocumentProcessor untuk memproses dokumen
DATA_PATH = "./data"  # Path folder yang berisi dokumen
extracted_docs = []  # List untuk menyimpan hasil ekstraksi dokumen
valid_extensions = ('.pdf', '.docx', '.txt')  # Jenis file yang didukung

# **Baca dan Proses Semua Dokumen**
all_texts = []  # Menyimpan semua teks hasil ekstraksi dari dokumen
metadata_list = []  # Menyimpan metadata untuk setiap teks
file_texts = {}  # Dictionary untuk menyimpan teks berdasarkan nama file

# Loop untuk membaca dan memproses semua dokumen dalam folder
for filename in os.listdir(DATA_PATH):
    if not filename.lower().endswith(valid_extensions):
        continue  # Skip file yang tidak memiliki ekstensi yang didukung

    filepath = os.path.join(DATA_PATH, filename)  # Buat path lengkap file
    with open(filepath, "rb") as f:  # Buka file dalam mode baca biner
        document = f.read()  # Baca konten file
        result = docs.process_document(document, filename)  # Proses dokumen menggunakan DocumentProcessor
        content, images, tables, plain_text = result  # Ambil hasil pemrosesan
        
        # **Simpan teks per file**
        file_texts[filename] = plain_text  # Simpan teks dalam dictionary berdasarkan nama file
        
        # **Split teks menjadi chunk**
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        chunks = text_splitter.split_text(plain_text)  # Bagi teks menjadi beberapa bagian (chunk)

        # **Tambahkan metadata (nama file sebagai sumber)**
        for chunk in chunks:
            all_texts.append(chunk)  # Simpan setiap chunk ke dalam list
            metadata_list.append({"source": filename})  # Simpan metadata (nama file sebagai sumber)

# Gunakan Hugging Face embeddings untuk merepresentasikan teks dalam bentuk vektor
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Simpan teks dan metadata dalam FAISS (vector store) untuk pencarian berbasis vektor
vectorstore = FAISS.from_texts(all_texts, embedding_model, metadatas=metadata_list)

# Konversi FAISS menjadi retriever untuk digunakan dalam pencarian informasi
retriever = vectorstore.as_retriever()


### 2. Inisialisasi Model dan Prompt untuk Ringkasan Teks

In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # Import template untuk membuat prompt chatbot
from langchain_ollama.llms import OllamaLLM  # Import LLM dari Ollama

# **Menentukan model Ollama yang digunakan**
OLLAMA_MODEL = "llama3.2"  # Model LLaMA 3.2 yang digunakan untuk pemrosesan bahasa alami
COLLECTION_NAME = "ollama_vectore_test"  # Nama koleksi yang digunakan dalam penyimpanan vektor

# **Membuat template prompt untuk tugas ringkasan teks**
template = """
  You are a helpful assistant for text summarization. 
  Only include information that is part of the document. 
  Do not include your own opinion or analysis.

  Document: 
  "{document}"
  Summary:
"""

# **Membuat instance prompt template dari teks di atas**
prompt = ChatPromptTemplate.from_template(template)

# **Menginisialisasi model LLaMA dari Ollama**
model = OllamaLLM(model=OLLAMA_MODEL)  # Membuat instance model untuk digunakan dalam tugas NLP


### 3. Retrieval QA Chain dan Fungsi Ringkasan

In [ ]:
# **Membuat Retrieval QA Chain** (Pencarian berbasis pertanyaan)
retrieval_chain = RetrievalQA.from_chain_type(
    llm=model,  # Menggunakan model LLaMA dari Ollama sebagai LLM
    retriever=retriever,  # Menggunakan retriever berbasis FAISS untuk mencari dokumen relevan
    chain_type="stuff",  # Menggunakan metode "stuff" untuk menyusun jawaban dari dokumen
    return_source_documents=True  # Mengembalikan dokumen sumber sebagai referensi jawaban
)

# **Fungsi untuk Menampilkan Jawaban dengan Sumber**
def retrieve_answer(query):
    response = retrieval_chain.invoke({"query": query})  # Menggunakan chain untuk menjawab query
    answer = response["result"]  # Mengambil hasil jawaban dari response
    
    # **Mengumpulkan sumber dokumen yang digunakan dalam jawaban**
    sources = {doc.metadata["source"] for doc in response["source_documents"]}
    
    # **Format sumber dokumen agar bisa ditampilkan dengan rapi**
    source_text = "\n\n**Sumber:**\n" + "\n".join(sources)
    
    return answer + source_text  # Mengembalikan jawaban beserta sumber dokumen

# **Fungsi untuk Meringkas Satu Dokumen**
def summarize_document(doc_name):
    if doc_name in file_texts:  # Cek apakah dokumen ada dalam sistem
        summary_prompt = f"Ringkas dokumen berikut:\n\n{file_texts[doc_name]}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)  # Menggunakan LLM untuk menghasilkan ringkasan
        
        return f"**Ringkasan dari {doc_name}:**\n{summary}"  # Mengembalikan hasil ringkasan
    else:
        return f"Dokumen '{doc_name}' tidak ditemukan dalam sistem."  # Pesan jika dokumen tidak ditemukan

# **Fungsi untuk Meringkas Semua Dokumen**
def summarize_all_documents(_=None):  # Tambahkan parameter dengan default None agar kompatibel dengan agent
    summaries = []  # List untuk menyimpan semua ringkasan dokumen
    
    for filename, text in file_texts.items():  # Iterasi setiap dokumen yang ada
        summary_prompt = f"Ringkas dokumen berikut:\n\n{text}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)  # Gunakan model untuk menghasilkan ringkasan
        
        summaries.append(f"**Ringkasan dari {filename}:**\n{summary}")  # Simpan ringkasan dalam list
    
    return "\n\n".join(summaries)  # Gabungkan semua ringkasan menjadi satu string yang rapi

# **Definisikan Tools untuk Agent**
tools = [
    Tool(
        name="retrieval_qa",
        func=retrieve_answer,  # Menghubungkan fungsi pencarian jawaban
        description="Gunakan ini untuk menjawab pertanyaan berdasarkan dokumen yang tersedia."
    ),
    Tool(
        name="summarize_document",
        func=summarize_document,  # Menghubungkan fungsi ringkasan satu dokumen
        description="Gunakan ini untuk meringkas satu dokumen. Masukkan nama file dokumen sebagai input."
    ),
    Tool(
        name="summarize_all_documents",
        func=summarize_all_documents,  # Menghubungkan fungsi ringkasan semua dokumen
        description="Gunakan ini untuk meringkas semua dokumen yang tersedia."
    )
]


### 4. Inisialisasi Agent dan Eksekusi Multi-Query

In [ ]:
# **Inisialisasi Agent dengan Multi-Query Handling**
agent = initialize_agent(
    tools=tools,  # Menggunakan tools yang telah didefinisikan sebelumnya (retrieval, summarization)
    llm=model,  # Menggunakan model LLaMA dari Ollama untuk pemrosesan bahasa alami
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Tipe agen yang memungkinkan pemrosesan query tanpa pelatihan khusus
    verbose=True,  # Mengaktifkan mode verbose untuk melihat detail eksekusi agen
    handle_parsing_errors=True  # Menangani error yang mungkin terjadi saat parsing input
)

# **Daftar Pertanyaan untuk Diuji oleh Agent**
queries = [
    "Ringkas semua dokumen tentang kesehatan mental."  # Pertanyaan yang akan dijawab oleh agen
]

# **Jalankan dan Tampilkan Hasil**
for query in queries:
    print(f"\n### Pertanyaan: {query}\n")  # Tampilkan pertanyaan yang sedang diproses
    
    # Agent akan memproses query dan memberikan respons
    response = agent.invoke({"input": query})  # Eksekusi query dengan agent
    
    # Ambil jawaban akhir (output) dari response
    final_answer = response.get("output", "Tidak ada jawaban yang dihasilkan.")  # Menangani jika tidak ada jawaban
    
    # Tampilkan jawaban akhir dalam format Markdown untuk pemformatan yang lebih baik
    display(Markdown(f"### Final Answer:\n\n{final_answer}"))



### Pertanyaan: Ringkas semua dokumen tentang kesehatan mental.



> Entering new AgentExecutor chain...
Thought: Aku harus menjawab pertanyaan ini dengan menggunakan alat "summarize_all_documents" untuk meringkas semua dokumen yang tersedia.

Action: summarize_all_documents
Action Input: None
Observation: **Ringkasan dari 22778-88667-1-PB.pdf:**
Artikel ini tidak memberikan ringkasan yang jelas tentang topik yang dibahas. Namun, berdasarkan judul dan beberapa kalimat yang saya baca, artikel ini mungkin membahas tentang pandemi COVID-19 dan dampaknya terhadap kesehatan mental di Indonesia.

Beberapa hal yang saya identifikasi sebagai topik yang dibahas adalah:

* Pandemi COVID-19 dan dampaknya terhadap kesehatan mental
* Kebijakan kesehatan mental di Indonesia dan tantangannya
* Dampak pandemi COVID-19 terhadap kebijakan kesehatan mental di Indonesia
* Pengaruh pandemi COVID-19 terhadap kesehatan mental masyarakat Indonesia

Namun, perlu diingat bahwa artikel ini tidak memberikan ring

### Final Answer:

Ringkasan dari semua dokumen tentang kesehatan mental di Indonesia adalah sebagai berikut:

Kesehatan mental di Indonesia masih memiliki beberapa tantangan, seperti stigma yang kuat terhadap penderita gangguan jiwa. Buku "Kesehatan Mental Masyarakat Indonesia" membahas pentingnya menjaga keseimbangan antara kehidupan fisik dan mental serta menghilangkan stigma yang memalukan bagi penderita gangguan jiwa.

Dalam beberapa tahun terakhir, pandemi COVID-19 telah menambah beban stres dan kecemasan pada masyarakat Indonesia, sehingga dampaknya terhadap kesehatan mental sangat signifikan. Penelitian "Analisis Kesehatan Mental Mahasiswa Perguruan Tinggi X Tahun Akaademik 2019/2020" menunjukkan bahwa mahasiswa masih mengalami stres, kecemasan dan depresi yang dapat mempengaruhi keseharaan dan kemampuan mereka.

Namun, ada beberapa upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di Indonesia. Buku "Kesehatan Mental Sumber Daya Manusia Indonesia" berisi daftar sumber dan judul penelitian yang terkait dengan topik kesehatan mental, serta beberapa paragraf yang membahas tentang pentingnya menjaga keseimbangan antara kehidupan fisik dan mental.

Dalam beberapa tahun terakhir, pemerintah Indonesia telah meningkatkan upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat. Contohnya adalah program "Kesehatan Mental Masyarakat" yang diluncurkan oleh Kementerian Kesehatan Republik Indonesia, yang bertujuan untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat.

Ringkasan dari semua dokumen tersebut menunjukkan bahwa kesehatan mental di Indonesia masih memiliki beberapa tantangan, tetapi ada juga upaya untuk meningkatkan kesadaran dan penanganan kesehatan mental di masyarakat.